In [1]:
#from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.compose         import ColumnTransformer
from sklearn.preprocessing   import OneHotEncoder, RobustScaler,FunctionTransformer
from sklearn.pipeline        import Pipeline, make_pipeline
from sklearn.dummy           import DummyRegressor
from sklearn.metrics         import mean_absolute_error
 
from xgboost import XGBRegressor

import datetime
import numpy as np

from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import QuantileTransformer
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
df_copy = df.copy()

features_copy = df_copy.drop(columns=['quantity'])
labels_copy   = df_copy.loc[:,'quantity']
features_copy[['day', 'month', 'year']] = pipe_pre.fit_transform(features_copy)[['day', 'month', 'year']].astype('category')

#mask_trainer = features_copy['date'].apply(lambda x: x.year)>2016
mask_trainer  = features_copy['year']==2016
#featyres_copy = features_copy.drop(columns=['date'])
if 'date' in features_copy.columns:
    features_copy = features_copy.drop(columns=['date'])
    
numeric_features = features_copy.select_dtypes(include=np.number).columns
categrc_features = features_copy.select_dtypes(include=['object','category']).columns

X_train_1, y_train_1 = features_copy[mask_trainer], labels_copy[mask_trainer]
X_test_1, y_test_1   = features_copy[~mask_trainer], labels_copy[~mask_trainer]


hgbdt_preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categrc_features),
        ("num", "passthrough", numeric_features),
    ],
    sparse_threshold=1,
    verbose_feature_names_out=False,
).set_output(transform="pandas")

hgbdt_model = make_pipeline(
    hgbdt_preprocessor,
    HistGradientBoostingRegressor(
        categorical_features=categrc_features, random_state=0
    ),
)

hgbdt_model.fit(X_train_1,y_train_1)

In [ ]:
common_params = {
    "subsample": 50,
    "n_jobs": 2,
    "grid_resolution": 20,
    "random_state": 0,
}
features_info = {
    # features of interest
    "features": numeric_features,
    # type of partial dependence plot
    "kind": "average",
    # information regarding categorical features
    "categorical_features": categrc_features,
}
_, ax = plt.subplots(ncols=2, nrows=2, figsize=(9, 8), constrained_layout=True)

display = PartialDependenceDisplay.from_estimator(
    hgbdt_model,
    X_train_1,
    **features_info,
    ax=ax,
    **common_params,
)
_ = display.figure_.suptitle(
    "Partial dependence del numero de ventas de bebidas",
    fontsize=16,
)
#result = permutation_importance(pipe_final_1, X_test, y_test, n_repeats=10, random_state=42)

In [ ]:
from sklearn.inspection import permutation_importance
import numpy as np

hgbdt_model.fit(X_train_1,y_train_1)
data = features_copy
result = permutation_importance(hgbdt_model, X_test_1, y_test_1, n_repeats=10, random_state=42)
result

In [ ]:
perm_sorted_idx = result.importances_mean.argsort()
plt.boxplot(
    result.importances[perm_sorted_idx].T,
    vert=False,
    labels=np.array(features_copy.columns)[perm_sorted_idx],
)
plt.title("Importancia de las Features por Incremento del Error")
plt.xlabel('Error')
plt.show()